In [6]:
import pandas  # used for importing the .csv files
import operator
import numpy as np # used for natural logs
import datetime # used for dates
hp_test = {} # set up a dictionary to hold the results of our optimization
for hp1 in range(2,3): # range to iterate over for the first hyperparameter, the scale 'k'
    for hp2 in (400,100): # range for the spread hyperparameter
        for hp3 in (1,2): # range for the effect homecourt advantage
            for hp4 in (0,-1): # range for the effect of playing in a back to back
                data = pandas.read_csv('games2012.csv') # read in csv
                ID = {} # set up a dictionary that will be the place of storage for teams and their ratings
                outcomes = [] # set up an array that will contain whether we correctly predicted the game or not
                for idx, row in data.iterrows(): # for each game....
                    if data.loc[idx,'PTS']>data.loc[idx,'PTS.1']: # check who won the game, this is the case where the away team won
                        winner = data.loc[idx,'Visitor/Neutral']; loser = data.loc[idx,'Home/Neutral']; # get the names of the winner and loser
                        pd = data.loc[idx,'PTS'] - data.loc[idx,'PTS.1']; hm_i = 0 #get the point differential and signal whether home team won
                    else: # do the same in the case where the home team won
                        winner = data.loc[idx,'Home/Neutral']; loser = data.loc[idx,'Visitor/Neutral'];
                        pd = data.loc[idx,'PTS.1'] - data.loc[idx,'PTS'];hm_i = 1
                    game_date = datetime.datetime.strptime(data.loc[idx]["Date"],"%a, %b %d, %Y") # get the date of the game
                    if winner in ID.keys(): # if the team has already played a game
                        w_elo = ID[winner][0] # get their ranking
                        if (ID[winner][1] - game_date).days == 1: w_date = 1 # check whether they are playing in a back-to-back game
                        else: w_date = 0 
                        w_games = ID[winner][2] # see how many previous games they've played
                    else: # else initialize a new team
                        w_elo = 1500
                        w_date = 0
                        w_games = 0
                    if loser in ID.keys(): #do the same for the losing team
                        l_elo = ID[loser][0]
                        if (ID[loser][1] - game_date).days == 1: l_date = 1
                        else: l_date = 0
                        l_games = ID[loser][2]
                    else: 
                        l_elo = 1500
                        l_date = 0
                        l_games = 0
                    if hm_i == 1: w_elo = w_elo+hp3 + hp4*w_date; l_elo = l_elo + hp4*l_date # adjust the teams' elo rating for homecourt advantage and back to back
                    else: l_elo = l_elo +hp3 + hp4*l_date; w_elo = w_elo + hp4*l_date;
                    if w_elo>l_elo: outcomes.append(1) # if the model correctly predicted the game, add a one to the outcome vector
                    if l_elo>w_elo: outcomes.append(0) # else add a zero
                    w_new_elo = (w_elo+np.log(pd+1)*(2.2/((w_elo-l_elo)*.001+2.2))*hp1*(1-(1/(1+10**((l_elo-w_elo)/400))))) # update the winner's elo
                    l_new_elo = (l_elo+np.log(pd+1)*(2.2/((w_elo-l_elo)*.001+2.2))*hp1*(0-(1/(1+10**((w_elo-l_elo)/400))))) # update the loser's elo
                    ID[winner]=(w_new_elo,game_date,w_games + 1) # update the dictionary of teams to include a key of the team's name, and a value of (their elo score, the date of their last game, how many games they've played so far)
                    ID[loser]=(l_new_elo,game_date,l_games + 1) # do the same for the losing team
                hp_test[(hp1,hp2,hp3,hp4)] = (sum(outcomes)/float(len(outcomes))) # update the dictionary of results to include a key of all their hyperparameters and a value of the percent of games correctly predicted
print (sorted(hp_test.items(), key=operator.itemgetter(1),reverse=True)) # print out the dictionary in order form most accurate to least accurate
print (sorted(ID.items()))

[((2, 400, 1, 0), 0.6323232323232323), ((2, 100, 1, -1), 0.6323232323232323), ((2, 100, 2, 0), 0.6323232323232323), ((2, 100, 1, 0), 0.6323232323232323), ((2, 100, 2, -1), 0.6323232323232323), ((2, 400, 1, -1), 0.6323232323232323), ((2, 400, 2, -1), 0.6323232323232323), ((2, 400, 2, 0), 0.6323232323232323)]
[('Atlanta Hawks', (1595.750378020135, datetime.datetime(2012, 4, 26, 0, 0), 66)), ('Boston Celtics', (1592.6989002444523, datetime.datetime(2012, 4, 26, 0, 0), 66)), ('Charlotte Bobcats', (1454.0151452155819, datetime.datetime(2012, 4, 26, 0, 0), 66)), ('Chicago Bulls', (1633.8860161822615, datetime.datetime(2012, 4, 26, 0, 0), 66)), ('Cleveland Cavaliers', (1508.5932694833098, datetime.datetime(2012, 4, 26, 0, 0), 66)), ('Dallas Mavericks', (1576.4034942958237, datetime.datetime(2012, 4, 26, 0, 0), 66)), ('Denver Nuggets', (1588.33871945695, datetime.datetime(2012, 4, 26, 0, 0), 66)), ('Detroit Pistons', (1529.6504170577762, datetime.datetime(2012, 4, 26, 0, 0), 66)), ('Golden Sta

In [42]:
print sorted(ID.items(), key=operator.itemgetter(1))

[('New York Knicks', 1429.1020865981511), ('Minnesota Timberwolves', 1429.8264966520815), ('Philadelphia 76ers', 1449.6889059015343), ('Los Angeles Lakers', 1452.8458649351214), ('Orlando Magic', 1481.041649560636), ('Sacramento Kings', 1505.7195279637463), ('Denver Nuggets', 1522.4209952449073), ('Charlotte Hornets', 1524.7164374552988), ('Detroit Pistons', 1547.3065579000677), ('Miami Heat', 1555.1196338937884), ('Brooklyn Nets', 1562.2676929516283), ('Phoenix Suns', 1566.935709631726), ('Milwaukee Bucks', 1581.4526295910337), ('Indiana Pacers', 1583.7306680480892), ('Boston Celtics', 1587.1195250155222), ('Washington Wizards', 1591.3868122903748), ('Utah Jazz', 1599.454372328689), ('New Orleans Pelicans', 1602.1887937926476), ('Toronto Raptors', 1612.6188414194928), ('Dallas Mavericks', 1624.9392707368092), ('Portland Trail Blazers', 1627.7719616507015), ('Oklahoma City Thunder', 1629.8439504046573), ('Chicago Bulls', 1630.0660556442467), ('Memphis Grizzlies', 1644.3861379004986), (

In [73]:
import datetime
import re
print -.1*((datetime.datetime.strptime(data.loc[1]["Date"],"%a, %b %d, %Y"))-(datetime.datetime.strptime(data.loc[6]["Date"],"%a, %b %d, %Y"))).days

0.1


In [2]:
3+4

7